In [4]:
import requests
import defs
import pandas as pd
import pickle
from tabulate import tabulate

In [5]:
session = requests.Session()

In [6]:
ins_df = pd.read_pickle("instruments.pkl")

In [7]:
ins_df

,name,item_type,displayName,pipLocation,marginRate
0,XAG_EUR,METAL,Silver/EUR,-4,0.10
1,CAD_SGD,CURRENCY,CAD/SGD,-4,0.05
2,XAG_USD,METAL,Silver,-4,0.10
3,UK100_GBP,CFD,UK 100,0,0.05
4,GBP_NZD,CURRENCY,GBP/NZD,-4,0.05
...,...,...,...,...,...
117,JP225_USD,CFD,Japan 225,0,0.05
118,XAG_SGD,METAL,Silver/SGD,-4,0.10
119,EUR_GBP,CURRENCY,EUR/GBP,-4,0.0333
120,XAG_HKD,METAL,Silver/HKD,-4,0.1


In [4]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [5]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params = params, headers = defs.SECURE_HEADER)
    return response.status_code, response.json()

In [6]:
def get_price(dictionary,prices,ohlc,candle):
    for price in prices:
        for oh in ohlc:
            dictionary[f"{price}_{oh}"] = candle[price][oh]

In [7]:
def get_candles_df(json_response):
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o','h','l','c']
    
    our_data= []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        price = get_price(new_dict,prices,ohlc,candle)
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [8]:
def get_combinations(our_curr):
    for first_idx in range(len(our_curr)):
        first_pair= our_curr[first_idx]
        for second_idx in range(0,len(our_curr)):
            second_pair = our_curr[second_idx]
            if(first_pair==second_pair):
                continue
            else:
                pair = f"{first_pair}_{second_pair}"
                print(pair)
        

In [9]:
code, res = fetch_candles("EUR_USD", 10, "H1")

In [10]:
df = get_candles_df(res)

In [11]:
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2021-05-05T06:00:00.000000000Z,4580,1.20122,1.20160,1.19932,1.19998,1.20114,1.20154,1.19925,1.19991,1.20129,1.20166,1.19938,1.20004
1,2021-05-05T07:00:00.000000000Z,5983,1.19998,1.20049,1.19860,1.19944,1.19992,1.20043,1.19853,1.19938,1.20005,1.20055,1.19867,1.19950
2,2021-05-05T08:00:00.000000000Z,4114,1.19946,1.19970,1.19871,1.19968,1.19940,1.19964,1.19864,1.19961,1.19951,1.19976,1.19878,1.19974
3,2021-05-05T09:00:00.000000000Z,2706,1.19966,1.20044,1.19932,1.20038,1.19959,1.20038,1.19925,1.20031,1.19973,1.20050,1.19938,1.20044
4,2021-05-05T10:00:00.000000000Z,4370,1.20038,1.20200,1.20038,1.20162,1.20032,1.20194,1.20032,1.20155,1.20045,1.20205,1.20044,1.20168
5,2021-05-05T11:00:00.000000000Z,3576,1.20162,1.20188,1.20068,1.20094,1.20157,1.20181,1.20062,1.20087,1.20168,1.20194,1.20074,1.20100
6,2021-05-05T12:00:00.000000000Z,6472,1.20092,1.20116,1.19916,1.19970,1.20086,1.20110,1.19909,1.19964,1.20099,1.20123,1.19922,1.19976
7,2021-05-05T13:00:00.000000000Z,6505,1.19969,1.20096,1.19944,1.19980,1.19963,1.20089,1.19938,1.19973,1.19975,1.20102,1.19950,1.19987
8,2021-05-05T14:00:00.000000000Z,6912,1.19983,1.20120,1.19937,1.20042,1.19976,1.20114,1.19931,1.20035,1.19990,1.20125,1.19943,1.20048


In [12]:
def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(f"{pair}_{granularity}.pkl")

In [13]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair,4000,granularity)
    if code != 200:
        print(pair,"Error")
        return
    df = get_candles_df(json_data)
    print(df)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df,pair,granularity)
    

In [14]:
create_data("EUR_USD","H1")

                                time  volume    mid_o    mid_h    mid_l  \
0     2020-09-11T00:00:00.000000000Z    2071  1.18236  1.18368  1.18236   
1     2020-09-11T01:00:00.000000000Z    2337  1.18351  1.18403  1.18312   
2     2020-09-11T02:00:00.000000000Z    1112  1.18333  1.18384  1.18314   
3     2020-09-11T03:00:00.000000000Z    1324  1.18332  1.18363  1.18303   
4     2020-09-11T04:00:00.000000000Z    1039  1.18339  1.18374  1.18319   
...                              ...     ...      ...      ...      ...   
3994  2021-05-05T10:00:00.000000000Z    4370  1.20038  1.20200  1.20038   
3995  2021-05-05T11:00:00.000000000Z    3576  1.20162  1.20188  1.20068   
3996  2021-05-05T12:00:00.000000000Z    6472  1.20092  1.20116  1.19916   
3997  2021-05-05T13:00:00.000000000Z    6505  1.19969  1.20096  1.19944   
3998  2021-05-05T14:00:00.000000000Z    6912  1.19983  1.20120  1.19937   

        mid_c    bid_o    bid_h    bid_l    bid_c    ask_o    ask_h    ask_l  \
0     1.18350  1.18

In [15]:
for first_index in range(len(our_curr)):
    first_pair = our_curr[first_index]
    for second_index in range(0,len(our_curr)):
        second_pair = our_curr[second_index]
        if(first_pair == second_pair):
            continue
        else:
            create_data(f"{first_pair}_{second_pair}", "H1")
        

                                time  volume    mid_o    mid_h    mid_l  \
0     2020-09-11T00:00:00.000000000Z    2071  1.18236  1.18368  1.18236   
1     2020-09-11T01:00:00.000000000Z    2337  1.18351  1.18403  1.18312   
2     2020-09-11T02:00:00.000000000Z    1112  1.18333  1.18384  1.18314   
3     2020-09-11T03:00:00.000000000Z    1324  1.18332  1.18363  1.18303   
4     2020-09-11T04:00:00.000000000Z    1039  1.18339  1.18374  1.18319   
...                              ...     ...      ...      ...      ...   
3994  2021-05-05T10:00:00.000000000Z    4370  1.20038  1.20200  1.20038   
3995  2021-05-05T11:00:00.000000000Z    3576  1.20162  1.20188  1.20068   
3996  2021-05-05T12:00:00.000000000Z    6472  1.20092  1.20116  1.19916   
3997  2021-05-05T13:00:00.000000000Z    6505  1.19969  1.20096  1.19944   
3998  2021-05-05T14:00:00.000000000Z    6912  1.19983  1.20120  1.19937   

        mid_c    bid_o    bid_h    bid_l    bid_c    ask_o    ask_h    ask_l  \
0     1.18350  1.18

                                time  volume    mid_o    mid_h    mid_l  \
0     2020-09-11T00:00:00.000000000Z    8525  1.77898  1.77960  1.77634   
1     2020-09-11T01:00:00.000000000Z    7738  1.77804  1.77848  1.77672   
2     2020-09-11T02:00:00.000000000Z    4339  1.77788  1.77817  1.77692   
3     2020-09-11T03:00:00.000000000Z    4672  1.77735  1.77763  1.77608   
4     2020-09-11T04:00:00.000000000Z    4103  1.77654  1.77664  1.77534   
...                              ...     ...      ...      ...      ...   
3994  2021-05-05T10:00:00.000000000Z   10626  1.67354  1.67442  1.67080   
3995  2021-05-05T11:00:00.000000000Z    8313  1.67131  1.67154  1.66972   
3996  2021-05-05T12:00:00.000000000Z   13946  1.67054  1.67067  1.66676   
3997  2021-05-05T13:00:00.000000000Z   14417  1.66684  1.66904  1.66464   
3998  2021-05-05T14:00:00.000000000Z   16969  1.66574  1.66804  1.66380   

        mid_c    bid_o    bid_h    bid_l    bid_c    ask_o    ask_h    ask_l  \
0     1.77804  1.77

                                time  volume    mid_o    mid_h    mid_l  \
0     2020-09-11T00:00:00.000000000Z    1000  1.31898  1.31950  1.31804   
1     2020-09-11T01:00:00.000000000Z     811  1.31858  1.31870  1.31756   
2     2020-09-11T02:00:00.000000000Z     412  1.31786  1.31799  1.31730   
3     2020-09-11T03:00:00.000000000Z     396  1.31783  1.31808  1.31745   
4     2020-09-11T04:00:00.000000000Z     311  1.31750  1.31759  1.31722   
...                              ...     ...      ...      ...      ...   
3994  2021-05-05T10:00:00.000000000Z     700  1.22851  1.22864  1.22686   
3995  2021-05-05T11:00:00.000000000Z     648  1.22740  1.22772  1.22692   
3996  2021-05-05T12:00:00.000000000Z    1269  1.22735  1.22822  1.22668   
3997  2021-05-05T13:00:00.000000000Z    1490  1.22772  1.22829  1.22680   
3998  2021-05-05T14:00:00.000000000Z    2123  1.22782  1.22832  1.22535   

        mid_c    bid_o    bid_h    bid_l    bid_c    ask_o    ask_h    ask_l  \
0     1.31860  1.31

                                time  volume    mid_o    mid_h    mid_l  \
0     2020-09-11T00:00:00.000000000Z    9611  1.92700  1.92700  1.92265   
1     2020-09-11T01:00:00.000000000Z    8321  1.92346  1.92499  1.92245   
2     2020-09-11T02:00:00.000000000Z    4939  1.92454  1.92520  1.92378   
3     2020-09-11T03:00:00.000000000Z    5081  1.92493  1.92586  1.92419   
4     2020-09-11T04:00:00.000000000Z    4912  1.92574  1.92586  1.92316   
...                              ...     ...      ...      ...      ...   
3994  2021-05-05T10:00:00.000000000Z   11573  1.93717  1.93836  1.93458   
3995  2021-05-05T11:00:00.000000000Z    8126  1.93504  1.93580  1.93372   
3996  2021-05-05T12:00:00.000000000Z   13896  1.93442  1.93516  1.93172   
3997  2021-05-05T13:00:00.000000000Z   15894  1.93209  1.93349  1.92804   
3998  2021-05-05T14:00:00.000000000Z   17301  1.92972  1.93168  1.92790   

        mid_c    bid_o    bid_h    bid_l    bid_c    ask_o    ask_h    ask_l  \
0     1.92345  1.92

                                time  volume   mid_o   mid_h   mid_l   mid_c  \
0     2020-09-11T01:00:00.000000000Z    3353  70.646  70.742  70.638  70.674   
1     2020-09-11T02:00:00.000000000Z    1776  70.672  70.724  70.665  70.700   
2     2020-09-11T03:00:00.000000000Z    1940  70.701  70.732  70.660  70.723   
3     2020-09-11T04:00:00.000000000Z    1875  70.722  70.778  70.719  70.750   
4     2020-09-11T05:00:00.000000000Z    1280  70.747  70.775  70.720  70.774   
...                              ...     ...     ...     ...     ...     ...   
3994  2021-05-05T10:00:00.000000000Z    4720  78.433  78.622  78.426  78.586   
3995  2021-05-05T11:00:00.000000000Z    3363  78.584  78.650  78.560  78.600   
3996  2021-05-05T12:00:00.000000000Z    7266  78.598  78.690  78.572  78.685   
3997  2021-05-05T13:00:00.000000000Z    8604  78.684  78.740  78.566  78.700   
3998  2021-05-05T14:00:00.000000000Z    9959  78.700  78.800  78.603  78.782   

       bid_o   bid_h   bid_l   bid_c   

                                time  volume    mid_o    mid_h    mid_l  \
0     2020-09-11T00:00:00.000000000Z     563  0.69018  0.69025  0.68986   
1     2020-09-11T01:00:00.000000000Z     388  0.69006  0.69068  0.68987   
2     2020-09-11T02:00:00.000000000Z     199  0.69059  0.69070  0.69050   
3     2020-09-11T03:00:00.000000000Z     174  0.69050  0.69061  0.69034   
4     2020-09-11T04:00:00.000000000Z     123  0.69060  0.69066  0.69052   
...                              ...     ...      ...      ...      ...   
3994  2021-05-05T10:00:00.000000000Z     541  0.74486  0.74502  0.74438   
3995  2021-05-05T11:00:00.000000000Z     406  0.74441  0.74450  0.74373   
3996  2021-05-05T12:00:00.000000000Z     752  0.74396  0.74464  0.74396   
3997  2021-05-05T13:00:00.000000000Z    1122  0.74406  0.74446  0.74317   
3998  2021-05-05T14:00:00.000000000Z    1317  0.74395  0.74530  0.74360   

        mid_c    bid_o    bid_h    bid_l    bid_c    ask_o    ask_h    ask_l  \
0     0.69005  0.69

In [16]:
pickle_off = open("CAD_CHF_H1.pkl","rb")
emp = pickle.load(pickle_off)
print(emp)

                                time  volume    mid_o    mid_h    mid_l  \
0     2020-09-11T00:00:00.000000000Z     563  0.69018  0.69025  0.68986   
1     2020-09-11T01:00:00.000000000Z     388  0.69006  0.69068  0.68987   
2     2020-09-11T02:00:00.000000000Z     199  0.69059  0.69070  0.69050   
3     2020-09-11T03:00:00.000000000Z     174  0.69050  0.69061  0.69034   
4     2020-09-11T04:00:00.000000000Z     123  0.69060  0.69066  0.69052   
...                              ...     ...      ...      ...      ...   
3994  2021-05-05T10:00:00.000000000Z     541  0.74486  0.74502  0.74438   
3995  2021-05-05T11:00:00.000000000Z     406  0.74441  0.74450  0.74373   
3996  2021-05-05T12:00:00.000000000Z     752  0.74396  0.74464  0.74396   
3997  2021-05-05T13:00:00.000000000Z    1122  0.74406  0.74446  0.74317   
3998  2021-05-05T14:00:00.000000000Z    1317  0.74395  0.74530  0.74360   

        mid_c    bid_o    bid_h    bid_l    bid_c    ask_o    ask_h    ask_l  \
0     0.69005  0.69

In [23]:
df = pd.read_data(emp[0])

AttributeError: module 'pandas' has no attribute 'read_data'